In [1]:
import random
from typing import Callable

from MutantTool import mutant_generator
from basicFunctions import xor, input_generator
from BV_MR3 import BV_Alg_MT, BV_MT3

from qiskit import QuantumCircuit, transpile, Aer
from qiskit.providers.aer import QasmSimulator

svsim = Aer.get_backend('aer_simulator')
simulator = QasmSimulator()

In [2]:
def mutation_MT(original_algorithm : Callable[[int],QuantumCircuit], MR_algorithm : Callable[[QuantumCircuit, [str]],QuantumCircuit], num_qubits: int, num_mutants : int, num_max_inputs=20, input_OnesComplement=False): 
    
    originalQC = original_algorithm(num_qubits)
    mutants = mutant_generator(originalQC, num_mutants)

    input_list = input_generator(num_qubits, num_max_inputs)
    if input_OnesComplement :
        aux = "1" * num_qubits
        input_pairs = [[input_list[i],xor(input_list[i],aux)]for i in range(len(input_list))]
    else:
        input_pairs = [[input_list[i],input_list[random.randint(0,len(input_list)-1)]]for i in range(len(input_list))]

    test = [MR_algorithm(x,y) for x in mutants for y in input_pairs]

    results = [svsim.run(transpile(x,svsim)).result().get_counts() for x in test]

    res_analysis = list(map(lambda x: (x['000']!=1024 if '000' in x.keys() else True),results))

    mutants_killed = sum(res_analysis)
    print("Mutants Killed: ",mutants_killed)
    
    print("Mutantion Score: ",mutants_killed/len(res_analysis)*100,"%")
    
# MutationScore is not real, it accounts if all inputs can kill the respective mutant.

In [3]:
algorithm = BV_Alg_MT
MR_algorithm = BV_MT3

num_qubits = 3
num_mutants = 5
num_max_inputs = 10

mutation_MT(algorithm,MR_algorithm, num_qubits, num_mutants, num_max_inputs)

Mutants Killed:  39
Mutantion Score:  97.5 %


In [4]:
mutation_MT(algorithm,MR_algorithm, num_qubits, num_mutants, num_max_inputs)

Mutants Killed:  37
Mutantion Score:  92.5 %


In [5]:
algorithm = BV_Alg_MT
MR_algorithm = BV_MT3

num_qubits = 4
num_mutants = 5
num_max_inputs = 10

mutation_MT(algorithm,MR_algorithm, num_qubits, num_mutants, num_max_inputs)

Mutants Killed:  50
Mutantion Score:  100.0 %


In [6]:
algorithm = BV_Alg_MT
MR_algorithm = BV_MT3

num_qubits = 5
num_mutants = 5
num_max_inputs = 10

mutation_MT(algorithm,MR_algorithm, num_qubits, num_mutants, num_max_inputs)

Mutants Killed:  50
Mutantion Score:  100.0 %


In [7]:
algorithm = BV_Alg_MT
MR_algorithm = BV_MT3

num_qubits = 3
num_mutants = 10
num_max_inputs = 10

mutation_MT(algorithm,MR_algorithm, num_qubits, num_mutants, num_max_inputs)

Mutants Killed:  63
Mutantion Score:  78.75 %
